In [ ]:
import os
import mysql.connector
from mysql.connector import errorcode
from inspect import getsourcefile

import pandas as pd
import typing as T
from pathlib import Path


def get_db_connection(host: str, username: str, password: str, dbname: str, port: int = 3306) -> T.Any:
    """
    Get access to a provided database.
    """
    try:
        return mysql.connector.connect(host=host, user=username, password=password, database=dbname, port=port)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your user name or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

def execute_sql_from_file(connection: T.Any, sql: T.Union[Path, str]) -> T.Any:
    """
    Read in a file from the provided filepath or string and execute the SQL contained in the file.
    """
    query = ""
    if isinstance(sql, Path):
        with open(sql, "r") as infile:
            query = infile.read()
        if not query:
            raise ValueError(f"File {sql.as_posix()} is empty.")
    else:
        query = sql

    db_cursor = connection.cursor()
    try:
        query_results = db_cursor.execute(query)
    except Exception as err:
        # close database connection before raising exception
        db_cursor.close()
        raise err

    return query_results


connection = get_db_connection("localhost", "root", "password", "movies")

In [ ]:
db_sql_path = Path().cwd().parent / "sql" / "create_tables.sql"

connection = get_db_connection("localhost", "root", "password", "movies")
data = execute_sql_from_file(connection, db_sql_path)

In [ ]:
db_sql_path = Path().cwd().parent / "sql" / "load_db.sql"

connection = get_db_connection("localhost", "root", "password", "movies")
data = execute_sql_from_file(connection, db_sql_path)

In [ ]:
data